# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

import nltk
# download libraries if necessary
# nltk.download(['punkt', 'stopwords', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [3]:
# load data from database
engine = create_engine('sqlite:///CleanDatabase.db')
df = pd.read_sql_table('cleaned_data', engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

In [27]:
y[y['related']==2]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
460,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20324,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20495,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22326,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23382,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """ Function to process text in the following order: normalize, tokenize, 
        remove stop words, and lemmatize.
    Args:
        text (str): text to be processed
    Returns:
        tokens (list): list of tokens after processing text
    """

    # normalize text by removing punctuation and converting to lower case
    text = re.sub(r'[^\w\s]', ' ', text.lower())

    # tokenize text
    tokens = word_tokenize(text)

    # remove stop words and lemmatize
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word)
              for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier

In [6]:
# build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
from sklearn.model_selection import train_test_split

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000020C3E6E0828>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [18]:
# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

In [21]:
from sklearn.metrics import classification_report

for column in y_test.columns:
    print(f'Category: {column}')
    print(classification_report(y_test[column], y_pred[column]))
    print('---------------------------------------------------------')
    print('\n')

Category: related
              precision    recall  f1-score   support

           0       0.59      0.33      0.43      1873
           1       0.81      0.92      0.86      5934
           2       0.36      0.33      0.34        58

    accuracy                           0.78      7865
   macro avg       0.59      0.53      0.54      7865
weighted avg       0.75      0.78      0.75      7865

---------------------------------------------------------


Category: request
              precision    recall  f1-score   support

           0       0.84      1.00      0.91      6533
           1       0.78      0.08      0.14      1332

    accuracy                           0.84      7865
   macro avg       0.81      0.54      0.53      7865
weighted avg       0.83      0.84      0.78      7865

---------------------------------------------------------


Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7694
           1       0.67      0.01      0.02       171

    accuracy                           0.98      7865
   macro avg       0.82      0.51      0.51      7865
weighted avg       0.97      0.98      0.97      7865

---------------------------------------------------------


Category: other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7450
           1       1.00      0.01      0.01       415

    accuracy                           0.95      7865
   macro avg       0.97      0.50      0.49      7865
weighted avg       0.95      0.95      0.92      7865

---------------------------------------------------------


Category: direct_report
              precision    recall  f1-score   support

           0       0.81      1.00      0.89      6321
           1       0.72      0.05      0.09      1544

    accuracy  

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters =

cv =

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.